In [ ]:
import io
import multiprocessing
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv',index_col=0)

In [ ]:
df.head()

In [ ]:
captions = list(df['caption'])

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string,strip_non_alphanum,strip_numeric

In [ ]:
class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        for file in captions:
            for line in open(f'{file}'):
                CUSTOM_FILTERS = [lambda x: x.lower(), remove_stopwords, strip_non_alphanum,strip_numeric]
                yield preprocess_string(line,CUSTOM_FILTERS)

In [ ]:
import gensim.models

In [ ]:
cores=multiprocessing.cpu_count() -1
cores

In [ ]:
sentences = MyCorpus()
model = gensim.models.Word2Vec(workers=cores,
                               window = 2,
                               vector_size=300,
                               min_count=6,
                               sample=1e-3)

In [ ]:
from time import time

In [ ]:
t = time()

model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
t = time()

model.train(sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
model.save('word2vec.fullmodel')